# Reactome data proposal

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#通路的节点连接
data = pd.read_csv('./data/ReactomePathwaysRelation_new_download.txt',sep = '\t',header=None)

In [4]:
#通路的特征信息（基因表示）
import re
def load_data_dict(filename):

    data_dict_list = []
    dict = {}
    with open( filename) as gmt:
        data_list = gmt.readlines()
        
        for row in data_list:
            genes = row.split('\t')
            
            genes = [ i.replace('\n','') for i in genes]
            dict[genes[1]] = genes[3:]

    return dict


gene_data = load_data_dict('./data/ReactomePathways.gmt')

In [4]:
#选出HSA的通路
data = data[data[0].str.contains('HSA')]  #选出HSA 的通路

In [5]:
#只取存在特征的通路节点
total_paths = list(set(gene_data.keys()))
len(total_paths)

2029

In [6]:
#删除不在通路的节点
data[3] = data[0].apply(lambda x : 1 if x in total_paths else 0 )
data[4] = data[1].apply(lambda x : 1 if x in total_paths else 0 )
data1 = data[data[3]==1]
data2= data1[data1[4]==1]

In [7]:
#初始化index
data2.reset_index(drop = True,inplace=True)

In [8]:
#构造临接矩阵
row , col = list(total_paths),list(total_paths)

paths_matrix = np.zeros([len(row),len(col)])

for  indexs in range(data2.shape[0]):
      r = data2.loc[indexs][0]
      c = data2.loc[indexs][1]
      pos_r = row.index(r)
      pos_c = col.index(c)
      paths_matrix[pos_r,pos_c] = 1

In [9]:
np.array(paths_matrix.nonzero())

array([[   0,    0,    0, ..., 2028, 2028, 2028],
       [ 136,  306,  544, ...,  548,  574, 1151]], dtype=int64)

In [10]:
from scipy import sparse
paths_matrix = sparse.csr_matrix(paths_matrix)  # array 转成 scipy.sparse

In [11]:
np.array(paths_matrix.nonzero())

array([[   0,    0,    0, ..., 2028, 2028, 2028],
       [ 136,  306,  544, ...,  548,  574, 1151]], dtype=int32)

# 构造特征矩阵

In [12]:
#获取图上节点的全部基因
total_gene = []
for index in total_paths:
     total_gene = total_gene + gene_data[index]  
    
total_gene_set = list(set(total_gene))
len(total_gene_set)

10690

In [13]:
mat = np.zeros((len(total_paths), len(total_gene_set))) #  生成矩阵 【行 列】
for p in total_paths:   
    gs =  gene_data[p]
    #  得到key 和 values  
    g_inds = [total_gene_set.index(g) for g in gs]   #   获取基因在矩阵的位置
    p_ind = total_paths.index(p)                      #   获取 通路在矩阵的位置
    mat[p_ind,g_inds] = 1

df = pd.DataFrame(mat, index=total_paths, columns=total_gene_set)

# 构建图数据集

In [14]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
#这里给出大家注释方便理解
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    #返回数据集源文件名
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    #返回process方法所需的保存文件名。你之后保存的数据集名字和列表里的一致
    @property
    def processed_file_names(self):
        return ['data.pt']
    
    
    #生成数据集
    def process(self):
        
        edge_index = torch.tensor(np.array(paths_matrix.nonzero()), dtype=torch.long)   #边的关系
 
        X = torch.tensor(df.values, dtype=torch.float)  #节点特征
    
        data = Data(x=X, edge_index=edge_index,)
    
        # Read data into huge `Data` list.
        data_list = [data]
 
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
 
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
 
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

D:\anaconda3\envs\GNN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
MyOwn =MyOwnDataset('ReactomeGCNData1')

Processing...
Done!


In [26]:
MyOwn.data

Data(x=[2029, 10690], edge_index=[2, 2003])

In [27]:
MyOwn.data.edge_index

tensor([[   0,    0,    0,  ..., 2028, 2028, 2028],
        [ 136,  306,  544,  ...,  548,  574, 1151]])

In [21]:
# print(row[2],row[1294])

In [28]:
import os.path as osp
import torch.nn.functional as F
import torch_geometric.transforms as T
from sklearn.metrics import roc_auc_score
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, train_test_split_edges

data = train_test_split_edges(MyOwn.data)


In [29]:
data

Data(x=[2029, 10690], val_pos_edge_index=[2, 50], test_pos_edge_index=[2, 100], train_pos_edge_index=[2, 1704], train_neg_adj_mask=[2029, 2029], val_neg_edge_index=[2, 50], test_neg_edge_index=[2, 100])

In [30]:
data.num_features = data.size(1)

In [31]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, 256)
        self.conv2 = GCNConv(256, 128)
        self.conv3 = GCNConv(128, out_channels)
        

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        return self.conv2(x, edge_index)

    def decode(self, z, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)  # [2,E]
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # *：element-wise乘法

    def decode_all(self, z):
        prob_adj = z @ z.t()  # @：矩阵乘法，自动执行适合的矩阵乘法函数
        return (prob_adj > 0).nonzero(as_tuple=False).t()

    def forward(self, x, pos_edge_index, neg_edge_index):
        return decode(encode(x, pos_edge_index), pos_edge_index, neg_edge_index)



In [32]:
def get_link_labels(pos_edge_index, neg_edge_index):
    num_links = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(num_links, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train(data, model, optimizer, criterion):
    model.train()

    neg_edge_index = negative_sampling(  # 训练集负采样，每个epoch负采样样本可能不同
        edge_index=data.train_pos_edge_index,
        num_nodes=data.num_nodes,
        num_neg_samples=data.train_pos_edge_index.size(1))

    optimizer.zero_grad()
    # link_logits = model(data.x, data.train_pos_edge_index, neg_edge_index)
    z = model.encode(data.x, data.train_pos_edge_index)
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index).to(data.x.device)  # 训练集中正样本标签
    loss = criterion(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss

In [33]:
@torch.no_grad()
def mytest(data,model):
    model.eval()

    z = model.encode(data.x, data.train_pos_edge_index)

    results = []
    for prefix in ['val', 'test']:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']
        link_logits = model.decode(z, pos_edge_index, neg_edge_index)
        link_probs = link_logits.sigmoid()#计算链路存在的概率
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        results.append(roc_auc_score(link_labels.cpu(), link_probs.cpu()))
    return results



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(data.num_features, 64).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = F.binary_cross_entropy_with_logits
best_val_auc = test_auc = 0


In [34]:
for epoch in range(1,100):
    loss=train(data,model,optimizer,criterion)
    val_auc,tmp_test_auc=mytest(data,model)
    if val_auc>best_val_auc:
        best_val_auc=val_auc
        test_auc=tmp_test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, Test: {test_auc:.4f}')
#预测
# z=model.encode(data.x,data.train_pos_edge_index)
# final_edge_index=model.decode_all(z)

Epoch: 001, Loss: 0.6160, Val: 0.7748, Test: 0.8126
Epoch: 002, Loss: 2.9047, Val: 0.5804, Test: 0.8126
Epoch: 003, Loss: 26.5050, Val: 0.8206, Test: 0.8399
Epoch: 004, Loss: 1.9961, Val: 0.8110, Test: 0.8399
Epoch: 005, Loss: 1.4101, Val: 0.6780, Test: 0.8399
Epoch: 006, Loss: 1.8808, Val: 0.6756, Test: 0.8399
Epoch: 007, Loss: 1.3100, Val: 0.6960, Test: 0.8399
Epoch: 008, Loss: 0.8006, Val: 0.7360, Test: 0.8399
Epoch: 009, Loss: 0.6899, Val: 0.7768, Test: 0.8399
Epoch: 010, Loss: 0.5457, Val: 0.7932, Test: 0.8399
Epoch: 011, Loss: 0.5435, Val: 0.7972, Test: 0.8399
Epoch: 012, Loss: 0.5374, Val: 0.8096, Test: 0.8399
Epoch: 013, Loss: 0.5410, Val: 0.8044, Test: 0.8399
Epoch: 014, Loss: 0.5179, Val: 0.8012, Test: 0.8399
Epoch: 015, Loss: 0.5142, Val: 0.7932, Test: 0.8399
Epoch: 016, Loss: 0.5023, Val: 0.7996, Test: 0.8399
Epoch: 017, Loss: 0.4919, Val: 0.8092, Test: 0.8399
Epoch: 018, Loss: 0.4823, Val: 0.8112, Test: 0.8399
Epoch: 019, Loss: 0.4876, Val: 0.8152, Test: 0.8399
Epoch: 020,

In [35]:
z=model.encode(data.x,data.train_pos_edge_index)

In [36]:
z

tensor([[ 0.0914,  0.2032,  0.7769,  ...,  0.0441, -0.4794, -0.3830],
        [-0.0371,  0.0172,  0.1125,  ...,  0.0162,  0.0194,  0.2207],
        [ 0.0092,  0.0266, -0.0223,  ...,  0.0091,  0.0148,  0.0192],
        ...,
        [-0.0439, -0.2785,  0.1426,  ...,  0.0477, -0.2052, -0.0135],
        [ 0.0959, -0.0035, -0.3102,  ...,  0.2329, -0.2712,  0.1266],
        [-0.0116, -0.0141, -0.0135,  ...,  0.1289,  0.3460,  0.1629]],
       grad_fn=<AddBackward0>)

In [37]:
new_path = pd.DataFrame(z.detach().numpy(),index = total_paths)

In [38]:
new_path

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
R-HSA-8953854,0.091381,0.203161,0.776924,-0.558911,0.284520,-0.140107,-0.000049,0.945102,0.168385,-0.723967,...,0.749231,0.265940,0.389738,-0.124046,-0.586530,0.507120,-0.437051,0.044124,-0.479429,-0.383033
R-HSA-211733,-0.037075,0.017154,0.112517,-0.254793,-0.153845,-0.117749,-0.028945,-0.074176,0.035665,0.101395,...,0.235917,-0.088024,-0.127737,0.056212,-0.013957,-0.029609,0.196879,0.016238,0.019418,0.220693
R-HSA-173599,0.009176,0.026553,-0.022261,-0.025813,-0.035659,-0.015939,0.005500,-0.016199,0.005756,0.013102,...,-0.010685,-0.037684,-0.025810,0.033827,0.002167,0.009618,-0.044949,0.009072,0.014811,0.019203
R-HSA-3560783,-0.255390,0.253245,-0.074782,0.317875,-0.370775,0.297177,-0.004499,-0.212764,0.356232,-0.118090,...,-0.264449,0.275320,-0.048571,0.248857,0.169930,0.108922,0.145072,-0.323338,0.266036,0.006522
R-HSA-140877,-0.215039,-0.202694,0.122047,-0.057728,-0.232950,-0.292914,-0.105633,-0.233615,-0.044733,-0.353424,...,-0.103033,-0.184272,0.109887,-0.487025,0.103150,0.227254,-0.299837,0.447151,-0.373352,-0.053435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-349425,-0.034888,0.031946,0.119353,-0.290362,-0.159712,-0.115350,-0.034734,-0.071284,0.028227,0.118884,...,0.271280,-0.087017,-0.125474,0.056010,-0.021660,-0.025991,0.226277,-0.001100,0.019383,0.255802
R-HSA-8934593,0.043711,0.018287,0.020465,-0.002057,-0.017925,-0.033605,0.047126,-0.057995,-0.027391,0.009594,...,0.022240,-0.023501,-0.009044,0.010507,0.077307,-0.003307,-0.040056,0.002286,0.004713,0.038609
R-HSA-6811434,-0.043857,-0.278549,0.142622,-0.003279,0.019841,-0.464538,-0.278462,-0.003130,-0.073984,0.204335,...,-0.457489,0.019473,-0.303331,0.211101,0.268194,-0.286352,-0.081349,0.047699,-0.205166,-0.013462
R-HSA-166665,0.095913,-0.003483,-0.310165,0.127688,0.053658,-0.117361,-0.044907,-0.101881,-0.178735,0.213908,...,-0.208097,-0.335245,-0.186561,-0.331478,0.203362,0.041079,0.248687,0.232874,-0.271221,0.126650


In [89]:
new_path.to_csv('./data/Pathways_Feature.csv',encoding='utf-8')

In [88]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z)) 
sigmoid(z[2].dot(z[1969]).detach().numpy())

0.9790995438476585